# **Transfer Learning with TensorFlow**

## **What is Transfer Learning?**

**Transfer Learning** reuses a **pre-trained model** for a new task.  
Instead of training from scratch, we adapt knowledge learned from a **large dataset** to a **smaller dataset**.

---

## **Model: `MobileNetV2`**

- Pre-trained on **`ImageNet`**
- Lightweight and efficient
- Suitable for **image classification**

---

## **Dataset: `CIFAR-10`**

- 60,000 images  
- 32 × 32 pixels  
- 10 classes  

---

## **Goal**

- Load **`MobileNetV2`**
- Replace the final layer
- Fine-tune on **`CIFAR-10`**
- Improve performance

---

## **Why Use Transfer Learning?**

- Faster training  
- Requires less data  
- Better accuracy  

Transfer Learning enables efficient and high-performing computer vision models.

# **`Step 1: Import Libraries and Load Data`**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values and convert labels to one-hot encoding
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# **Data Augmentation**

## **What is Data Augmentation?**

**Data Augmentation** increases the **size** and **diversity** of training data by creating modified versions of existing data without collecting new samples.

Common techniques (for images):

- **Rotation**
- **Flipping**
- **Scaling / Zooming**
- **Cropping**
- **Adding Noise**
- **Brightness Adjustment**

---

## **Why It Matters**

- **Prevents Overfitting** – Improves generalization  
- **Boosts Performance** – More diverse training data  
- **Handles Imbalance** – Augments minority classes  

> Always choose augmentations that make sense for your task (e.g., avoid flipping text images).

---

# **Step 2: Data Preprocessing**

## **Image Size Adjustment**

- **`MobileNetV2`** expects: **224 × 224**
- **`CIFAR-10`** images are: **32 × 32**

### **Required Steps**

- **Resize to 224 × 224**
- Apply **Data Augmentation**
- **Normalize / Rescale** pixel values

Proper **Augmentation + Preprocessing** ensures compatibility with **`MobileNetV2`** and improves model performance.

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True ,
    vertical_flip = True,
    zoom_range = 0.1,
    fill_mode = 'nearest'
)

datagen.fit(x_train)

# Step 3: Modify the Pre-Trained Model

## Why Modify the Model?

MobileNetV2 is pre-trained on ImageNet with 1000 classes.  
Our task (CIFAR-10) has only 10 classes, so we must:

- Remove the original top layer  
- Add a new classifier for 10 classes  
- Freeze the base layers (initially)

---

## What We Do

### 1. Load MobileNetV2 Without Top Layer
- `include_top=False`
- Keep feature extraction layers
- Remove the 1000-class classifier

### 2. Add New Classifier
- Global Average Pooling
- Dense layer with 10 output units
- Softmax activation

We use fewer classes (10 instead of 1000).

### 3. Freeze Base Layers
- Pre-trained weights are not updated
- Only the new classifier trains first
- Later fine-tuning can be applied